In [327]:
import pandas as pd
import cvxpy as cp
import pickle
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations, combinations_with_replacement, product
import math

In [328]:
# Run the Semidefinite program

from tabnanny import verbose


def run_localization_sdp(n, availPairs, dist_err, pd_mat):
    # gen the M_ij for the constraints
    Mij_storage = []
    for i,j in availPairs:
        eij = np.zeros((n,1))
        eij[i] = 1
        eij[j] = -1
        mij = np.matmul(eij, np.transpose(eij))
        Mij_storage.append(mij)


    X = cp.Variable((n,n), symmetric=True)
    constraints = [X >> 0] # X must be Positive Semi Definite
    constraints += [
        cp.trace(Mij_storage[i] @ X) - (pd_mat[availPairs[i,0], availPairs[i,1]] ** 2) <= dist_err for i in range(p)
    ]
    constraints += [
        cp.trace(Mij_storage[i] @ X) - (pd_mat[availPairs[i,0], availPairs[i,1]] ** 2) >= -dist_err for i in range(p)
    ]

    prob = cp.Problem(cp.Minimize(cp.trace(X)),
                    constraints)
    prob.solve()

    return X

In [329]:
# Eigen decomposition of the XX^T Mat we got as X
def get_final_points_rep(Q, num_dim):
    w,v = np.linalg.eig(Q)
    w = np.real(w)

    # sort these based on the eigenvalues
    v = v[:,np.argsort(-w)]
    w = w[np.argsort(-w)]

    # best rank 'd' approximation, keep only the first d elements
    w[num_dim:] = 0
    # getting the best 'd' dimensional approximation of the points
    final_points_rep = np.matmul(v, np.diag(np.sqrt(w)))[:,:num_dim]

    return final_points_rep

In [330]:
def get_best_context_vec(availPairs, true_rep, final_points_rep, num_dim, plot_title=None):
    points_considered = np.unique(np.ravel(availPairs))
    _n = points_considered.shape[0]
    centering_mat = np.eye(_n) - np.ones((_n,_n))/_n

    # original points
    orig_reps = true_rep[points_considered, :] # * context_vectors[(0,1)]
    est_reps = final_points_rep[points_considered, :]

    # context search
    count = 0
    contexts = []
    metrics = []
    for roll in product([0,1], repeat = num_dim):
        reps_in_context = orig_reps * np.array(roll)
        d_x_xHat =  np.matmul(est_reps, est_reps.transpose()) - \
                    np.matmul(reps_in_context, reps_in_context.transpose())
        d_x_xHat = np.matmul(centering_mat, np.matmul(d_x_xHat, centering_mat))

        contexts.append(roll)
        metrics.append( np.linalg.norm(np.ravel(d_x_xHat), 1)/ (n **2) )

    # plt.plot(metrics)
    # if plot_title != None: plt.title('context = ' + plot_title)
    # plt.show()

    best_context = contexts[np.argmin(metrics)]
    # print(best_context)
    # print(np.min(metrics))

    return best_context

In [331]:
with open("est_dist.pkl", "rb") as inptr:
    est_dist = pickle.load(inptr)
with open("true_rep.pkl", "rb") as inptr:
    true_rep = np.load(inptr)

In [332]:
all_contexts = list(combinations_with_replacement([0, 1, 2], r=3))
print(all_contexts)

[(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 1), (0, 1, 2), (0, 2, 2), (1, 1, 1), (1, 1, 2), (1, 2, 2), (2, 2, 2)]


In [333]:
context_dist = {i : {} for i in all_contexts}
for key, vals in est_dist.items():
    for key2, vals2 in vals.items():
        context_dist[key2][key] = vals2

In [334]:
# params for the semidefinite program
# max err in the estimated and measured distances
dist_err = 0.5
# num dims for the final points rep
num_dim = 2 # this can be atmost 'n'
n = len(true_rep)

In [335]:
_labels = np.array([1, 2, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 0, 2, 2, 1, 0, 0, 1, 2, 1, 1,
       0, 2, 1, 0, 2, 2, 2, 1, 1, 0, 0])
cluster_to_image_index = {
    0: list(np.where(_labels == 0)[0]),
    1: list(np.where(_labels == 1)[0]),
    2: list(np.where(_labels == 2)[0]),
}

In [336]:
best_context_vecs = {}
find_reps_from_config = {}
find_reps_truncated_from_config = {}

for context in all_contexts:
    pd_mat = np.zeros((n,n))
    availPairs = np.array(list(context_dist[context].keys()))
    p = len(availPairs)
    availPoints = np.unique(np.ravel(availPairs))
    for key, val in context_dist[context].items():
        i,j = key
        pd_mat[i,j] = val
        pd_mat[j,i] = val

    X = run_localization_sdp(n, availPairs, dist_err, pd_mat)
    final_points_rep = get_final_points_rep(X.value, num_dim)

    best_context_vec = get_best_context_vec(availPairs, true_rep, final_points_rep, num_dim, str(context))
    best_context_vecs[context] = best_context_vec
    find_reps_from_config[context] = final_points_rep
    find_reps_truncated_from_config[context] = {i: final_points_rep[i] for i in availPoints}

/var/folders/36/1dhnptp91hzb49wm7_3_1b6m0000gn/T/ipykernel_21044/1274202443.py:13: RuntimeWarning: invalid value encountered in sqrt
  final_points_rep = np.matmul(v, np.diag(np.sqrt(w)))[:,:num_dim]


In [337]:
print('Estimated Context Vectors')
for key, val in best_context_vecs.items():
    print(key,':', val) 

Estimated Context Vectors
(0, 0, 0) : (0, 1)
(0, 0, 1) : (1, 0)
(0, 0, 2) : (0, 1)
(0, 1, 1) : (1, 0)
(0, 1, 2) : (1, 0)
(0, 2, 2) : (0, 1)
(1, 1, 1) : (0, 0)
(1, 1, 2) : (1, 0)
(1, 2, 2) : (1, 0)
(2, 2, 2) : (0, 0)


In [338]:
find_reps_from_point = {}
for config, point_to_rep in find_reps_truncated_from_config.items():
    for point, rep in point_to_rep.items():
        context = best_context_vecs[config]
        if point not in find_reps_from_point:
            find_reps_from_point[point] = {context: rep}
        else:
            find_reps_from_point[point][context] = rep

In [339]:
find_rep_cleaned_from_point = {}
for point, context_to_rep in find_reps_from_point.items():
    count_1_occur = np.zeros(num_dim)
    accumu = np.zeros(num_dim)
    for context, rep_arr in context_to_rep.items():
        ctx_arr = np.array(context)
        count_1_occur += ctx_arr
        accumu += np.real(rep_arr) * ctx_arr
    mean_at_1 = accumu / count_1_occur
    cleaned_mean_at_1 = np.nan_to_num(mean_at_1)

    find_rep_cleaned_from_point[point] = cleaned_mean_at_1

/var/folders/36/1dhnptp91hzb49wm7_3_1b6m0000gn/T/ipykernel_21044/175719899.py:9: RuntimeWarning: invalid value encountered in divide
  mean_at_1 = accumu / count_1_occur


In [340]:
sorted_by_point = list(map(lambda x: x[1], sorted(find_rep_cleaned_from_point.items(), key=lambda x: x[0])))

# Result with context

In [341]:
from sklearn.cluster import KMeans
import numpy as np

X = np.array(sorted_by_point)
kmeans_context = KMeans(n_clusters=3, random_state=0).fit(X)
print("K means ")
kmeans_context.labels_

K means 


array([0, 1, 0, 2, 0, 0, 2, 1, 0, 0, 2, 0, 1, 2, 0, 1, 1, 2, 1, 2, 1, 2,
       0, 1, 1, 1, 0, 1, 2, 0, 2, 2, 2], dtype=int32)

In [342]:
true_labels = np.array([1, 2, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 0, 2, 2, 1, 0, 0, 1, 2, 1, 1,
       0, 2, 1, 0, 2, 2, 2, 1, 1, 0, 0])

In [343]:
def vi(predicted_labels,
       true_labels):
    def expand(labels):
        expanded = {label: set() for label in labels}
        for node, label in enumerate(labels):
            expanded[label].add(node)
        return list(expanded.values())

    n = len(predicted_labels)
    predicted = expand(predicted_labels)
    p = list(map(lambda x: len(x) / n, predicted))
    true = expand(true_labels)
    q = list(map(lambda x: len(x) / n, true))
    r = [[len(predicted[i].intersection(true[j])) / n
          for j in range(len(true))]
         for i in range(len(predicted))]
    vi = sum([r[i][j] * (np.log(r[i][j] / p[i]) + np.log(r[i][j] / q[j]))
              if r[i][j] > 0 else 0
              for j in range(len(true))
              for i in range(len(predicted))]) * -1
    return abs(vi)

In [344]:
vi(kmeans_context.labels_, true_labels)

2.1801193174215676

# Result without considering context

In [345]:
avg_dist = {k: np.mean(list(v.values())) for k, v in est_dist.items()}

In [346]:
pd_mat = np.zeros((n,n))
availPairs = np.array(list(avg_dist.keys()))
p = len(availPairs)
for key, val in avg_dist.items():
    i,j = key
    pd_mat[i,j] = val
    pd_mat[j,i] = val

X = run_localization_sdp(n, availPairs, dist_err, pd_mat)
final_points_rep = get_final_points_rep(X.value, num_dim)

In [347]:
X = final_points_rep
kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
kmeans.labels_

array([0, 1, 0, 2, 0, 0, 2, 1, 0, 0, 2, 0, 1, 2, 0, 1, 1, 2, 1, 2, 1, 2,
       0, 1, 1, 1, 0, 1, 2, 0, 2, 2, 2], dtype=int32)

In [348]:
vi(kmeans.labels_, true_labels)

2.1801193174215676

# Next

In [349]:
kmeans_context.labels_

np.save("new_labels.npy", kmeans_context.labels_)